# Graph Lab

## Header information:

  - Author #1: Evan Placenis (placenie@mcmaster.ca)
  - Author #2: Jad Haytaoglu (email@mcmaster.ca)
  - Gitlab URL: http://gitlab.cas.mcmaster.ca/haytaogj/l1-graph-lab
  - Avenue to Learn group name: Graph 45

In [7]:
# from matplotlib import pyplot as plt
# import csv

# with open('_dataset/london.connections.csv') as file:  
#     connections = csv.reader(file)

# for line in connections:
    


SyntaxError: incomplete input (4142502672.py, line 8)

In [12]:
# class AdjNode:
#     def __init__(self, value):
#         self.vertex = value
#         self.next = None


# class Graph:
#     def __init__(self, num):
#         self.V = num
#         self.graph = [None] * self.V

#     # Add edges
#     def add_edge(self, s, d):
#         node = AdjNode(d)
#         node.next = self.graph[s]
#         self.graph[s] = node

#         node = AdjNode(s)
#         node.next = self.graph[d]
#         self.graph[d] = node

#     # Print the graph
#     def print_agraph(self):
#         for i in range(self.V):
#             print("Vertex " + str(i) + ":", end="")
#             temp = self.graph[i]
#             while temp:
#                 print(" -> {}".format(temp.vertex), end="")
#                 temp = temp.next
#             print(" \n")

# if __name__ == "__main__":
#     V = 5

#     # Create graph and edges
#     graph = Graph(V)
#     graph.add_edge(0, 1)
#     graph.add_edge(0, 2)
#     graph.add_edge(0, 3)
#     graph.add_edge(1, 2)

#     graph.print_agraph()

Vertex 0: -> 3 -> 2 -> 1 

Vertex 1: -> 2 -> 0 

Vertex 2: -> 1 -> 0 

Vertex 3: -> 0 

Vertex 4: 



In [15]:
# adj_list = {}
# mylist = []
# def add_node(node):
#   if node not in mylist:
#     mylist.append(node)
#   else:
#     print("Node ",node," already exists!")
 
# def add_edge(node1, node2):
#   temp = []
#   if node1 in mylist and node2 in mylist:
#     if node1 not in adj_list:
#       temp.append(node2)
#       adj_list[node1] = temp
   
#     elif node1 in adj_list:
#       temp.extend(adj_list[node1])
#       temp.append(node2)
#       adj_list[node1] = temp
       
#   else:
#     print("Nodes don't exist!")
 
# def graph():
#   for node in adj_list:
#     print(node, " ---> ", [i for i in adj_list[node]])
 
# #Adding nodes
# add_node(0)
# add_node(1)
# add_node(2)
# add_node(3)
# add_node(4)
# #Adding edges
# add_edge(0,1)
# add_edge(1,2)
# add_edge(2,3)
# add_edge(3,0)
# add_edge(3,4)
# add_edge(4,0)
 
# #Printing the graph
# graph()
 
# #Printing the adjacency list
# print(adj_list)

0  --->  [1]
1  --->  [2]
2  --->  [3]
3  --->  [0, 4]
4  --->  [0]
{0: [1], 1: [2], 2: [3], 3: [0, 4], 4: [0]}


In [18]:
#Undirected, weighted graph!!

class Graph:
    def __init__(self, num_of_nodes, directed=False):
        self.m_num_of_nodes = num_of_nodes
        self.m_nodes = range(self.m_num_of_nodes)

        # Define the type of a graph
        self.m_directed = directed

        self.m_adj_list = {node: set() for node in self.m_nodes}      

    def add_edge(self, node1, node2, weight):
        self.m_adj_list[node1].add((node2, weight))
        
        if not self.m_directed:
        	self.m_adj_list[node2].add((node1, weight))

    def print_adj_list(self):
        for key in self.m_adj_list.keys():
            print("node", key, ": ", self.m_adj_list[key])



# graph = Graph(5)

# graph.add_edge(0, 0, 25)
# graph.add_edge(0, 1, 5)
# graph.add_edge(0, 2, 3)
# graph.add_edge(1, 3, 1)
# graph.add_edge(1, 4, 15)
# graph.add_edge(4, 2, 7)
# graph.add_edge(4, 3, 11)

# graph.print_adj_list()

node 0 :  {(2, 3), (0, 25), (1, 5)}
node 1 :  {(3, 1), (4, 15)}
node 2 :  set()
node 3 :  set()
node 4 :  {(2, 7), (3, 11)}


In [42]:
from matplotlib import pyplot as plt
import csv

with open('_dataset/london.stations.csv') as file:  
    stations = csv.reader(file)

    #row count other than first line of headers is number of stations
    numStations = sum(1 for row in stations)-1
    
    graph = Graph(numStations)

    with open('_dataset/london.connections.csv') as file2:
        connections = csv.reader(file2)
        for fileLine in connections:
            # print(fileLine)
            if connections.line_num != 1:
                # print(fileLine)
                graph.add_edge(int(fileLine[0]),int(fileLine[1]),int(fileLine[3]))

    
    # graph.print_adj_list() #prints only 301 lines but should be 303?
        

    

node 0 :  set()
node 1 :  {(73, 2), (234, 4), (265, 3)}
node 2 :  {(263, 4), (156, 2)}
node 3 :  {(263, 2), (156, 4), (295, 2)}
node 4 :  {(70, 2), (201, 2)}
node 5 :  {(194, 3), (252, 2)}
node 6 :  {(46, 4)}
node 7 :  {(145, 2), (188, 3)}
node 8 :  {(124, 3), (264, 2)}
node 9 :  {(31, 3), (232, 3)}
node 10 :  {(95, 2), (128, 1)}
node 11 :  {(249, 4), (94, 6), (104, 3), (83, 3), (28, 2), (163, 1), (212, 2)}
node 12 :  {(257, 1), (56, 2)}
node 13 :  {(279, 4), (167, 3), (250, 2), (225, 2), (156, 2), (157, 2)}
node 14 :  {(92, 1), (167, 2)}
node 15 :  {(78, 4), (269, 2)}
node 16 :  {(91, 2), (173, 2)}
node 17 :  {(293, 2), (110, 1), (74, 3), (110, 2)}
node 18 :  {(193, 2), (186, 2)}
node 19 :  {(97, 2)}
node 20 :  {(65, 2), (217, 2)}
node 21 :  {(67, 3), (269, 2)}
node 22 :  {(111, 2), (47, 2)}
node 23 :  {(157, 3), (41, 2)}
node 24 :  {(164, 2), (156, 3)}
node 25 :  {(255, 2), (161, 1)}
node 26 :  {(274, 2), (260, 2)}
node 27 :  {(79, 2), (201, 2)}
node 28 :  {(107, 2), (162, 1), (192, 

In [ ]:
class Itinerary:
    def __init__(self,graph,station1,station2):
        self.graph = graph
        self.station1 = station1
        self.station2 = station2

    def rankPaths(self,path1,path2):
        #if length(path1) == length(path2):
        #   whichever has less transfers
        #otherwise: if shortestPath(path1,path2) == path1: return path1 else return path2

    def getShortestPath(self,path1,path2):
        TODO #Dijikstra's

    def numTransfers(self,other):
        TODO
